In [31]:
print(torch.cuda.is_available())
torch.cuda.get_device_name(0)
torch.cuda.device_count()

/opt/conda/lib/python3.8/site-packages/torch/cuda/__init__.py:146: UserWarning: 
Graphics Device with CUDA capability sm_86 is not compatible with the current PyTorch installation.
The current PyTorch install supports CUDA capabilities sm_37 sm_50 sm_60 sm_70.
If you want to use the Graphics Device GPU with PyTorch, please check the instructions at https://pytorch.org/get-started/locally/

  warnings.warn(incompatible_device_warn.format(device_name, capability, " ".join(arch_list), device_name))


4

In [32]:
torch.__version__

# import torchvision
torchvision.__version__

'0.8.2+cu110'

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
random_seed = 1
torch.manual_seed(random_seed)

## torch 기본 리뷰

In [16]:
data = torch.FloatTensor([[1,2,3,4,5,6,7,8]])
data.shape

torch.Size([1, 8])

In [17]:
data.view(2,2,-1)[0]

tensor([[1., 2.],
        [3., 4.]])

In [25]:
data.unsqueeze(2).shape 

torch.Size([1, 8, 1])

In [23]:
data.squeeze(0).shape

torch.Size([8])

## torch 의 학습 과정
1. 데이터 준비
2. 모델 초기화
3. optimizer 설정
4. 학습
    1. 예측 (model.train() 모드 설정)
    2. cost 함수 설정
    3. 최적화 계산

In [ ]:
### Simple Linear Regression
### y = w*x + b
# 1) 데이터 준비
x_train = torch.FloatTensor([[1],[2],[3]])
y_train = torch.FloatTensor([[2],[4],[6]])

# 2) 모델 초기화 : 각 parameter 를 0으로 초기화
W = torch.zeros(1, requires_grad = True) 
# requires_grad는 autograd 에 모든 연산(operation)들을 추적
b = torch.zeros(1, requires_grad = True)

# 3) 최적화 설정
optimizer = optim.SGD([W,b], lr =0.1)

# 4) 학습
total_epoch = 20 # 총 학습 횟수
for epoch in range(total_epoch):
    # 예측
    hypothesis = x_train*W +b
    # cost 함수 설정(MSE)
    cost = torch.mean((hypothesis - y_train)**2)
    
    # 최적화 계산
    optimizer.zero_grad() 
    # torch에서 gradient 정보가 남아있기 때문에 초기화를 시켜야 한다
    cost.backward()
    optimizer.step()
    
    print(f'Epoch : {epoch+1:2d}/{total_epoch} \
          W : {W.item():.4f}, b: {b.item():.4f}, \
          cost : {cost.item() : .6f}')

In [ ]:
### use nn module
# 1) 데이터 준비
x_train = torch.FloatTensor([[1],[2],[3]])
y_train = torch.FloatTensor([[2],[4],[6]])

# 2) 모델설정
class SimpleLinearRegression(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(1,1) # w의 개수, 결과 값의 개수
        
    def forward(self, x):
        return self.linear(x)

model = SimpleLinearRegression()

# 3) 최적화 설정
optimizer = optim.SGD(model.parameters(), lr=0.1)

# 4) 학습
total_epoch = 20
for epoch in range(total_epoch):
    hypothesis = model(x_train)
    cost = F.mse_loss(hypothesis, y_train) # 직접 구하지 않고 funtional 사용
    
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    print(f'Epoch : {epoch+1:2d}/{total_epoch} \
          W : {W.item():.4f}, b: {b.item():.4f}, \
          cost : {cost.item() : .6f}')

In [ ]:
### classification
from sklearn.datasets import load_iris

# 1) 데이터 준비
x_data = torch.FloatTensor(load_iris().data) # 그냥 tensor 를 쓰면 타입 오류가 발생함
y_data = torch.tensor(load_iris().target,)

# 2) 모델 생성
class MultiClassifier(nn.Module):
    def __init__(self, w = 4, b = 3): # 컬럼 개수 4개, 결과 값 3개
        super().__init__()
        self.linear = nn.Linear(w, b)
        
    def forward(self, x):
        return self.linear(x)

model = MultiClassifier()

# 3) optimizer
optimizer = optim.Adam(model.parameters(), lr=0.1)

# 4) 학습
epochs = 1000
model.train() # 훈련모드 
for epoch in range(epochs+1):
    
    # 1) 예측
    hypothesis = model(x_data)
    
    # 2) cost 
    cost = F.cross_entropy(hypothesis, y_data) # softmax + nll_loss : cross_entropy, binary class 일 경우 binary_cross_entropy 사용 
    
    # 3) 최적화 계산
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    # 로그출력
    if epoch % 100 == 0:
        print(f'Epoch : {epoch:4d} Cost : {cost.item():.6f}')